In [16]:
# This script aims to perform rotation detection by checking the rotation, speed features etc.
# The performance is not good for this way.
# Jincheng Pang 08/26/2015

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import scipy.signal as sg


# Converted from Dmitri's R code for rotation calcuation
# Added some parts for speed calcuation
def speeds_calculation(df):
    nrow,ncol = df.shape
    df['y'] = df.Angle*np.pi/180
    df['s'] = 0 
    # Large angle change between adjacent frames might be caused from +/- pi flucuation 
    for jj in xrange(1,nrow):
        df.loc[jj,'s'] = df.loc[jj-1,'s']
        diffValue = df.loc[jj,'y']-df.loc[jj-1,'y']
        diffPos = max(abs(df.loc[jj,'Cx']-df.loc[jj-1,'Cx']),abs(df.loc[jj,'Cy']-df.loc[jj-1,'Cy']))
        if np.abs(diffValue) <= 2:
            continue
        
        if diffValue > 2:
            df.loc[jj,'y'] = np.pi - df.loc[jj,'y']
        elif diffValue < -2:
            df.loc[jj,'s'] = df.loc[jj,'s'] + np.pi
            
        if diffPos > 30:
            df.loc[jj,'Cx'] = df.loc[jj-1,'Cx']
            df.loc[jj,'Cy'] = df.loc[jj-1,'Cy']
            
    # Calculate speed 
    df['vec'] = 0
    df['dist'] = 0
    
    df.loc[range(1,nrow),'vec'] = np.sqrt(np.square(np.diff(df.Cx))+np.square(np.diff(df.Cy)))
    df.loc[range(1,nrow),'dist'] = np.cumsum(df.loc[range(1,nrow),'vec'])
    
    df['rot'] = df.s + df.y
    
    return df

def generate_annotation(indNum,fs):
    indNum = indNum/fs
    hourNum = int(indNum//3600)
    indNum = indNum%3600
    minNum = int(indNum//60)
    indNum = indNum%60
    secNum = int(math.floor(indNum))
    miniSecNum = int((indNum-secNum)*100)
    return str(hourNum).zfill(2), str(minNum).zfill(2),str(secNum).zfill(2),str(miniSecNum).zfill(2)

featureFileName = 'C:\\PostDoctorProjects\\VideoEEGData\\Shank2_20150609\\video_stitched\\Shan2_20150609_out_bottom.csv'
df = pd.read_csv(featureFileName)

print generate_annotation(15*60+15*1,15.0)

('00', '01', '01', '00')


In [174]:
def rotation_detection(df1):

    nfr = np.array(range(df1.shape[0]))

    segLen = int(fs*4)
    segNum = nfr//segLen

    rotationSeg = [0]*nfr
    det_df = []

    speed_th = np.pi/15
    rot_th = 2
    inter_th = 6
    pos_th = 1.5
    size_th = df1.quantile(0.1,'1')['cnt_area']

    for ii in range(segNum[-1]-1):
    #for ii in range(2):
        segInd1 = (segNum==ii)
        segRot1 = df1.loc[segInd1,'rot']
        minRot1 = np.min(segRot1)
        minInd1 = np.argmin(segRot1)
        maxRot1 = np.max(segRot1)
        maxInd1 = np.argmax(segRot1)
        #diffPos = max(abs(df1.Cx[maxInd1]-df1.Cx[minInd1]),abs(df1.Cy[maxInd1]-df1.Cy[minInd1]))
        diffPos = np.median(df1.vec[segInd1])
        ratSize = np.min(df1.cnt_area[segInd1])

        if ratSize>size_th and (maxRot1-minRot1)/float(abs(maxInd1-minInd1))> speed_th and maxRot1-minRot1> rot_th and abs(maxInd1-minInd1)> inter_th and diffPos<pos_th:
            rotationSeg[segInd1] = 1
            hh1,mm1,ss1,ll1 = generate_annotation(ii*segLen+df1.ifr[0],fs)
            hh2,mm2,ss2,ll2 = generate_annotation((ii+1)*segLen-1+df1.ifr[0],fs)
            startTime =  hh1 + ':' + mm1 + ':' + ss1 + ':' + ll1 
            endTime =  hh2 + ':' +  mm2 + ':' + ss2 + ':' + ll2 

            if startTime > endTime:
                startTime, endTime = endTime, startTime

            item = [startTime, endTime, 'rotation']
            if item not in det_df:
                det_df.append(item)
            continue


        segInd2 = (segNum==ii+1)
        segRot2 = df1.loc[segInd2,'rot']
        minRot2 = np.min(segRot2)
        minInd2 = np.argmin(segRot2)
        maxRot2 = np.max(segRot2)
        maxInd2 = np.argmax(segRot2)

        maxRot = max(maxRot1,maxRot2)
        minRot = min(minRot1,minRot2)  

        if maxRot==maxRot2 and minRot ==  minRot2:
            continue

        if maxRot==maxRot1:
            maxInd = maxInd1
        else:
            maxInd = maxInd2

        if minRot==minRot1:
            minInd = minInd1
        else:
            minInd = minInd2

        if maxInd< minInd:
            maxInd, minInd = minInd,maxInd

        #diffPos = max(abs(df1.Cx[maxInd]-df1.Cx[minInd]),abs(df1.Cy[maxInd]-df1.Cy[minInd]))
        #diffPos = max(np.max(df1.vec[segInd1]),np.max(df1.vec[segInd2]))
        diffPos = max(np.median(df1.vec[segInd1]),np.median(df1.vec[segInd2]))
        # print minInd,maxInd
        ratSize = min(np.min(df1.cnt_area[segInd1]),np.min(df1.cnt_area[segInd2]))

        if ratSize>size_th and (maxRot-minRot)/float(maxInd-minInd)> speed_th and (maxRot-minRot)> rot_th and abs(maxInd-minInd)> inter_th and diffPos<pos_th:
            rotationSeg[segInd1] = 1
            rotationSeg[segInd2] = 1
            hh1,mm1,ss1,ll1 = generate_annotation(ii*segLen+df1.ifr[0],fs)
            hh2,mm2,ss2,ll2 = generate_annotation((ii+2)*segLen-1+df1.ifr[0],fs)
            startTime =  hh1 + ':' + mm1 + ':' + ss1 + ':' + ll1 
            endTime =  hh2 + ':' +  mm2 + ':' + ss2 + ':' + ll2 


            item = [startTime, endTime, 'rotation']
            if item not in det_df:
                det_df.append(item)
            #print preStartTime + ' ' + preEndTime + ' ' + 'rotation ' # + str(maxRot-minRot)


    for item in det_df:
        print item[0], item[1], item[2]

In [17]:
df1 = df[df['animal']=='73-1']
df1 = df1.reset_index(drop=True)

df2 = df[df['animal']=='77-5']
df2 = df2.reset_index(drop=True)

df3 = df[df['animal']=='73-4']
df3 = df3.reset_index(drop=True)

df4 = df[df['animal']=='88-2']
df4 = df4.reset_index(drop=True)

df5 = df[df['animal']=='77-3']
df5 = df5.reset_index(drop=True)

df6 = df[df['animal']=='73-6']
df6 = df6.reset_index(drop=True)

df7 = df[df['animal']=='87-7']
df7 = df7.reset_index(drop=True)

print max(df1.Angle), min(df1.Angle)

179.9845123 0.003968519


In [18]:
tic = time.time()
df1 = speeds_calculation(df1)
toc = time.time()
print toc-tic
tic = time.time()
df2 = speeds_calculation(df2)
toc = time.time()
print toc-tic
tic = time.time()
df3 = speeds_calculation(df3)
toc = time.time()
print toc-tic
tic = time.time()
df4 = speeds_calculation(df4)
toc = time.time()
print toc-tic
tic = time.time()
df5 = speeds_calculation(df5)
toc = time.time()
print toc-tic
tic = time.time()
df6 = speeds_calculation(df6)
toc = time.time()
print toc-tic
tic = time.time()
df7 = speeds_calculation(df7)
toc = time.time()
print toc-tic

45.5429999828
43.0370001793
48.1519999504
46.3480000496
49.6569998264
57.0150001049
51.4560000896


In [132]:
fs = 15.0
nfr = range(1000,1500)
plt.subplot(7,4,1)
plt.plot(df1.ifr[nfr]/fs/60,df1.y[nfr]/np.pi*180,'r')
plt.plot(df1.ifr[nfr]/fs/60,df1.s[nfr],'b')
plt.plot(df1.ifr[nfr]/fs/60,df1.Angle[nfr],'k')
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df1.loc[0,'animal'])
plt.subplot(7,4,2)
plt.plot(df1.ifr[nfr]/fs/60,df1.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df1.loc[0,'animal'])
plt.subplot(7,4,3)
plt.plot(df1.ifr[nfr]/fs/60,df1.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df1.loc[0,'animal'])
plt.subplot(7,4,4)
plt.plot(df1.ifr[nfr]/fs/60,df1.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df1.loc[0,'animal'])

plt.subplot(7,4,5)
plt.plot(df2.ifr[nfr]/fs/60,df2.y[nfr]/np.pi*180)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df2.loc[0,'animal'])
plt.subplot(7,4,6)
plt.plot(df2.ifr[nfr]/fs/60,df2.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df2.loc[0,'animal'])
plt.subplot(7,4,7)
plt.plot(df2.ifr[nfr]/fs/60,df2.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df2.loc[0,'animal'])
plt.subplot(7,4,8)
plt.plot(df2.ifr[nfr]/fs/60,df2.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df2.loc[0,'animal'])

plt.subplot(7,4,9)
plt.plot(df3.ifr[nfr]/fs/60,df3.y[nfr]/np.pi*180)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df3.loc[0,'animal'])
plt.subplot(7,4,10)
plt.plot(df3.ifr[nfr]/fs/60,df3.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df3.loc[0,'animal'])
plt.subplot(7,4,11)
plt.plot(df3.ifr[nfr]/fs/60,df3.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df3.loc[0,'animal'])
plt.subplot(7,4,12)
plt.plot(df3.ifr[nfr]/fs/60,df3.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df3.loc[0,'animal'])

plt.subplot(7,4,13)
plt.plot(df4.ifr[nfr]/fs/60,df4.y[nfr]/np.pi*180)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df4.loc[0,'animal'])
plt.subplot(7,4,14)
plt.plot(df4.ifr[nfr]/fs/60,df4.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df4.loc[0,'animal'])
plt.subplot(7,4,15)
plt.plot(df4.ifr[nfr]/fs/60,df4.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df4.loc[0,'animal'])
plt.subplot(7,4,16)
plt.plot(df4.ifr[nfr]/fs/60,df4.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df4.loc[0,'animal'])

plt.subplot(7,4,17)
plt.plot(df5.ifr[nfr]/fs/60,df5.y[nfr]/np.pi*180)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df5.loc[0,'animal'])
plt.subplot(7,4,18)
plt.plot(df5.ifr[nfr]/fs/60,df5.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df5.loc[0,'animal'])
plt.subplot(7,4,19)
plt.plot(df5.ifr[nfr]/fs/60,df5.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df5.loc[0,'animal'])
plt.subplot(7,4,20)
plt.plot(df5.ifr[nfr]/fs/60,df5.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df5.loc[0,'animal'])

plt.subplot(7,4,21)
plt.plot(df6.ifr[nfr]/fs/60,df6.y[nfr]/np.pi*180)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df6.loc[0,'animal'])
plt.subplot(7,4,22)
plt.plot(df6.ifr[nfr]/fs/60,df6.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df6.loc[0,'animal'])
plt.subplot(7,4,23)
plt.plot(df6.ifr[nfr]/fs/60,df6.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df6.loc[0,'animal'])
plt.subplot(7,4,24)
plt.plot(df6.ifr[nfr]/fs/60,df6.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df6.loc[0,'animal'])

plt.subplot(7,4,25)
plt.plot(df7.ifr[nfr]/fs/60,df7.y[nfr]/np.pi*180)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df7.loc[0,'animal'])
plt.subplot(7,4,26)
plt.plot(df7.ifr[nfr]/fs/60,df7.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df7.loc[0,'animal'])
plt.subplot(7,4,27)
plt.plot(df7.ifr[nfr]/fs/60,df7.vec[nfr]*fs)
plt.xlabel("Minutes")
plt.ylabel("Speed")
plt.title(df7.loc[0,'animal'])
plt.subplot(7,4,28)
plt.plot(df7.ifr[nfr]/fs/60,df7.dist[nfr])
plt.xlabel("Minutes")
plt.ylabel("Distance")
plt.title(df7.loc[0,'animal'])

plt.show()
df1.describe()


,ifr,cnt_area,Cx,Cy,cnt_solidity,Eccentricity,Angle,y,s,vec,dist,rot
count,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000,26475.000000
mean,30637.000000,1368.778924,386.831305,283.630072,0.904750,0.869266,91.155033,1.342074,224.661431,1.838725,25040.723227,226.003506
std,7642.818525,174.521751,22.569547,22.524924,0.043697,0.062364,51.903926,0.876775,116.154880,2.093659,13674.478780,116.213513
min,17400.000000,771.500000,335.325623,236.963592,0.476663,0.217851,0.003969,0.000069,0.000000,0.000000,0.000000,0.004079
25%,24018.500000,1292.000000,366.338470,264.942749,0.885831,0.846465,46.330433,0.510988,125.663706,0.564758,14103.071111,126.833272
50%,30637.000000,1367.500000,391.006409,280.889893,0.915242,0.883234,93.279243,1.324175,235.619449,1.255874,24909.409140,237.595186
75%,37255.500000,1451.000000,405.166122,302.751175,0.935542,0.910818,135.592102,2.069681,307.876080,2.456534,36832.847602,309.617603
max,43874.000000,5639.500000,437.587799,342.661591,0.974184,0.970688,179.984512,3.141291,449.247749,85.950089,48680.245210,452.130550


In [179]:
#print 'rat 1'
rotation_detection(df1)

#print 'rat 2'
rotation_detection(df2)

#print 'rat 3'
rotation_detection(df3)

#print 'rat 4'
rotation_detection(df4)

#print 'rat 5'
rotation_detection(df5)

#print 'rat 6'
rotation_detection(df6)

#print 'rat 7'
rotation_detection(df7)


00:20:36:00 00:20:39:93 rotation
00:21:36:00 00:21:39:93 rotation
00:23:24:00 00:23:27:93 rotation
00:23:36:00 00:23:39:93 rotation
00:25:24:00 00:25:27:93 rotation
00:27:32:00 00:27:35:93 rotation
00:30:24:00 00:30:27:93 rotation
00:34:20:00 00:34:23:93 rotation
00:35:20:00 00:35:23:93 rotation
00:43:04:00 00:43:07:93 rotation
00:46:44:00 00:46:47:93 rotation
00:57:08:00 00:57:11:93 rotation
00:57:36:00 00:57:39:93 rotation
01:00:36:00 01:00:39:93 rotation
01:05:36:00 01:05:39:93 rotation
01:33:39:06 01:33:43:00 rotation
01:35:43:06 01:35:47:00 rotation
02:16:34:06 02:16:42:00 rotation
02:28:50:06 02:28:54:00 rotation
03:03:41:06 03:03:45:00 rotation
03:10:13:06 03:10:17:00 rotation
03:10:17:06 03:10:21:00 rotation
03:10:53:06 03:10:57:00 rotation
03:11:29:06 03:11:33:00 rotation
03:11:41:06 03:11:45:00 rotation
03:59:29:00 03:59:32:93 rotation
04:23:10:00 04:23:13:93 rotation
04:23:34:00 04:23:37:93 rotation
04:25:42:00 04:25:45:93 rotation


In [32]:
'''
nfr = np.array(range(df1.shape[0]))
import peakdet
#nfr = np.array(range(480*15,820*15))
 
maxtab, mintab = peakdet.peakdet(np.diff(df1.rot[nfr]),1.0)
ind =  np.array(maxtab,dtype=int)[:,0]+1


df1['rotFrame'] = 0
df1.loc[nfr[ind],'rotFrame'] = 1
df1.head(10)
df1.to_csv('df1_rotation_detection.csv')
#print(df1.ifr[nfr[ind]])

plt.subplot(5,1,1)
plt.plot(df1.ifr[nfr]/fs,df1.rot[nfr]/2/np.pi)
plt.xlim(df1.ifr[nfr[0]]/fs,df1.ifr[nfr[-1]]/fs)
plt.title('Rotation')
plt.grid()

plt.subplot(5,1,2)
plt.plot(df1.ifr[nfr[1:]]/fs,np.diff(df1.rot[nfr]))
plt.scatter(df1.ifr[nfr[ind]]/fs,np.array(maxtab[:,1]))   
plt.xlim(df1.ifr[nfr[1]]/fs,df1.ifr[nfr[-1]]/fs)
plt.title('Derivative of rotation')
plt.grid()

plt.subplot(5,1,3)
plt.plot(df1.ifr[nfr]/fs,df1.Angle[nfr])
plt.xlim(df1.ifr[nfr[0]]/fs,df1.ifr[nfr[-1]]/fs)
plt.title('Angle')
plt.grid()
plt.subplot(5,1,4)
plt.plot(df1.ifr[nfr[1:]]/fs,np.diff(df1.Angle[nfr]))
plt.title('Derivative of angle (angle speed)')
plt.xlim(df1.ifr[nfr[1]]/fs,df1.ifr[nfr[-1]]/fs)
plt.grid()
plt.subplot(5,1,5)
plt.plot(df1.ifr[nfr[1:]]/fs,np.diff(df1.dist[nfr]))
plt.xlim(df1.ifr[nfr[1]]/fs,df1.ifr[nfr[-1]]/fs)
plt.title('speed)')
plt.grid()

plt.show()
'''

KeyboardInterrupt: 